# Парсинг

In [2]:
import requests  
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import time
import pandas as pd
import re
import numpy as np

In [3]:
def get_soupec(url: str) -> BeautifulSoup:
    response = requests.get(url)
    return BeautifulSoup(response.content, 'html.parser')

def get_grid(tree: BeautifulSoup):
    return tree.find_all('div', {'class': 'catalog-grid__item'})

 Соберем информацию, которая находится на карточках товаров в каталоге.

In [4]:
def get_bottle_info(bottle):
    if bottle.find('a', {'class': 'product-snippet__name js-dy-slot-click'}) == None:
        return
    else:
        href = bottle.find('a', {'class': 'product-snippet__name js-dy-slot-click'}).get('href')
        name = bottle.find('a', {'class': 'product-snippet__name js-dy-slot-click'}).text.replace('\xa0','').strip()
        w_id = int(bottle.find('div', {'class': 'product-snippet__id'}).text.split('\n')[2].strip())
        
        if bottle.find('div', {'class': 'product-snippet__price product-snippet__price_discount'}) == None:
            base_price = int(bottle.find('div', {'class': 'product-snippet__price'}).text.replace('₽','').replace(' ', ''))
            discount_size = None
            discount_price = None
        else:
            base_price = int(bottle.find('div', {'class': 'product-snippet__discount'}).text.split('\n')[1].replace('₽','').replace(' ', ''))
            discount_size = -int(bottle.find('div', {'class': 'product-snippet__discount'}).text.split('\n')[2].replace('%',''))
            discount_price = int(bottle.find('div', {'class': 'product-snippet__price product-snippet__price_discount'}).text.replace('₽','').replace(' ', ''))
        
        simple_collection = (bottle.find('span', {'class': 'product-snippet__simple-collection'}) != None)
        
        basic_info = bottle.find('div', {'class': 'product-snippet__desc'}).find_all('a')
        names = [str(i).replace('<a href="/catalog/vino/filter/', '').split('-')[0] for i in basic_info]
        values = [str(i).split('">')[1].replace('</a>','').replace('\xa0','') for i in basic_info]
        basic_info_dict = dict(zip(names,values))
        
        try:
            country = basic_info_dict['country']
        except KeyError:
            country = None
        try:
            color = basic_info_dict['color']
        except KeyError:
            color = None
        try:  
            sweetness = basic_info_dict['sugar_type']
        except KeyError:
            sweetness = None
        try:
            volume = float(basic_info_dict['volume'].replace('л.',''))
        except KeyError:
            volume = None
        
        no_alco = bottle.find('div', {'class': 'product-snippet__ratings-noalco'}) != None
        
        if bottle.find_all('div', {'class': 'product-snippet__ratings-item'}) == None:
            SW_rating = None
            VIVINO_rating = None
            WS_rating = None
            RVF_rating = None
            JS_rating = None
            RP_rating = None
            AM_rating = None
            GR_rating = None
            AIS_rating = None
            PENIN_rating = None
            ST_rating = None
            JR_rating = None
        else:
            ratings = dict([rating.text.split('\n')[1:3]  for rating in bottle.find_all('div', {'class': 'product-snippet__ratings-item'})])
            all_rat_keys = ['VIVINO', 'WS', 'RVF', 'JS', 'RP', 'AM', 'GR', 'AIS', 'PENIN', 'ST', 'JR']
            
            if list(set(ratings.keys()).difference(all_rat_keys)):
                SW_rating = float(list(set(ratings.keys()).difference(all_rat_keys))[0].strip())
            else:
                SW_rating = None
            try:
                VIVINO_rating = float(ratings.get('VIVINO'))
            except TypeError:
                VIVINO_rating = None
            try:
                WS_rating = float(ratings.get('WS'))
            except TypeError:
                WS_rating = None
            try:
                RVF_rating = float(ratings.get('RVF'))
            except TypeError:
                RVF_rating = None
            try:
                JS_rating = float(ratings.get('JS'))
            except TypeError:
                JS_rating = None
            try:
                RP_rating = float(ratings.get('RP'))
            except TypeError:
                RP_rating = None
            try:
                AM_rating = float(ratings.get('AM'))
            except TypeError:
                AM_rating = None
            try:
                GR_rating = float(ratings.get('GR'))
            except TypeError:
                GR_rating = None
            try:
                AIS_rating = float(ratings.get('AIS'))
            except TypeError:
                AIS_rating = None
            try:
                PENIN_rating = float(ratings.get('PENIN'))
            except TypeError:
                PENIN_rating = None
            try:
                ST_rating = float(ratings.get('ST'))
            except TypeError:
                ST_rating = None
            try:
                JR_rating = float(ratings.get('JR'))
            except TypeError:
                JR_rating = None
            
    return {'name': name,
            'w_id': w_id,
            'href': href,
            'base_price': base_price,
            'discount_size': discount_size,
            'discount_price': discount_price,
            'simple_collection': simple_collection,
            'country': country,
            'color': color,
            'sweetness': sweetness,
            'volume': volume,
            'no_alco': no_alco,
            'SW_rating': SW_rating,
            'VIVINO_rating': VIVINO_rating,
            'WS_rating': WS_rating,
            'RVF_rating': RVF_rating,
            'JS_rating': JS_rating,
            'RP_rating': RP_rating,
            'AM_rating': AM_rating,
            'GR_rating': GR_rating,
            'AIS_rating': AIS_rating,
            'PENIN_rating': PENIN_rating,
            'ST_rating': ST_rating,
            'JR_rating': JR_rating}

In [5]:
url = 'https://simplewine.ru/catalog/vino/page'
alco_data = []
print('Будет сделано, шеф!')
for p in tqdm(range(1, 178)):
    alco_data.extend([get_bottle_info(i) for i in get_grid(get_soupec(url+str(p)))])
    time.sleep(0.05)
alco_data_result = list(filter(None, alco_data))
print('Готово!')

Будет сделано, шеф!


  0%|          | 0/177 [00:00<?, ?it/s]

Готово!


Скопируем на всякий случай, чтобы ничего не испортить, так как следующая часть потребует намного больше времени.

In [6]:
alco_data_result_copy = alco_data_result.copy()

Напишем функцию, которая будет перехзодить по ссылкам с карточек и собирать всю оставшуюся информацию в наш словарик.

In [16]:
def add_info(bottle):
    soup = get_soupec('https://simplewine.ru/'+bottle['href'])
    
    brief_info = soup.find('dl', {'class': 'product-brief'})
    try:
        names = brief_info.text.split('\n\n')[:-1:2]
        names = [i.replace('\n','').replace(':','') for i in names]
        values = brief_info.text.split('\n\n')[1::2]
        brief_info_dict = dict(zip(names, values))
    except AttributeError:
        brief_info_dict = {}
    
    try:
        bottle['grape'] = brief_info_dict.get('Виноград').replace('\xa0','').split(',')
    except AttributeError:
        bottle['grape'] = None
    try:
        bottle['manufacturer'] = brief_info_dict.get('Производитель')
    except TypeError:
        bottle['manufacturer'] = None
    try:
        bottle['gift_wrap'] = (brief_info_dict.get('Подарочная упаковка') == 'Да')
    except TypeError:
        bottle['gift_wrap'] = False
    
    features_info = soup.find('ul', {'class': 'product-types__list'})
    if features_info != None:
        bottle['features'] = re.sub(r'([А-Я])', r' \1', features_info.text.replace('\n','').replace(' ','')).split()
    else:
        bottle['features'] = None
    try:
        more_info = soup.find('dl', {'class': 'characteristics-params__list'}).text.split('\n\n\n\n')
        try:
            more_info_dict = dict([i.replace('\n', '').strip().split(':') for i in more_info])
        except ValueError:
            more_info_dict = {}
    except AttributeError:
        more_info_dict = {}
    
    try:
        bottle['region'] = more_info_dict.get('Регион').strip()
    except AttributeError:
        bottle['region'] = None
    try:
        bottle['year'] = more_info_dict.get('Год').strip()
    except AttributeError:
        bottle['year'] = None
    try:
        bottle['strength'] = float(more_info_dict.get('Крепость').strip().replace('%',''))
    except AttributeError:
        bottle['strength'] = None
    try:
        bottle['storage_potential'] = more_info_dict.get('Потенциал хранения').strip()
    except AttributeError:
        bottle['storage_potential'] = None
    try:
        bottle['appellation'] = more_info_dict.get('Аппелласьон').strip()
    except AttributeError:
        bottle['appellation'] = None
    try:
        bottle['category'] = more_info_dict.get('Категория').strip()
    except AttributeError:
        bottle['category'] = None
    try:
        bottle['decantation'] = more_info_dict.get('Декантация').strip()
    except AttributeError:
        bottle['decantation'] = None
    try:
        bottle['aging_in_container'] = more_info_dict.get('Выдержка в емкости').strip()
    except AttributeError:
        bottle['aging_in_container'] = None
    try:
        tags_names = [i.text.split('\n\n')[0].replace('\n','') for i in soup.find_all('div', {'class': 'product-tags__block'})]
        tags_items = [[r.strip() for r in i.text.split('\n\n\n')[1].split('\n\n')[:-1]] for i in soup.find_all('div', {'class': 'product-tags__block'})]
        tags = dict(zip(tags_names, tags_items))
    except ValueError:
        tags = {}
    try:
        bottle['taste'] = tags.get('Вкус')
    except AttributeError:
        bottle['taste'] = None
    try:
        bottle['dishes'] = tags.get('Гастрономия')
    except AttributeError:
        bottle['dishes'] = None
    try:
        taste_names = [i.text.replace('\n','').strip() for i in soup.find_all('li', {'class': 'taste__item'})]
        taste_numbers = [(np.array([str(i) for i in list(r)[1:-1]]) == '<div class="taste__dot taste__dot_active"></div>').sum() for r in soup.find_all('div', {'class': 'taste__dots'})]
        dots = dict(zip(taste_names, taste_numbers))
    except TypeError:
        dots = {}
    try:
        bottle['sugar'] = dots.get('Сладость')
    except AttributeError:
        bottle['sugar'] = None
    try:
        bottle['acidity'] = dots.get('Кислотность')
    except AttributeError:
        bottle['acidity'] = None
    try:
        bottle['aromaticity'] = dots.get('Ароматичность')
    except AttributeError:
        bottle['aromaticity'] = None
    try:
        bottle['tannins'] = dots.get('Танины')
    except AttributeError:
        bottle['tannins'] = None
    try:
        bottle['body'] = dots.get('Тело')
    except AttributeError:
        bottle['body'] = None
    try:
        bottle['description'] = soup.find('div', {'class': 'characteristics-description'}).text.split('\n\n\n\n')[1:]
    except AttributeError:
        bottle['description'] = None

In [82]:
tags_names = [i.text.split('\n\n')[0].replace('\n','') for i in soup.find_all('div', {'class': 'product-tags__block'})]
tags_items = [[r.strip() for r in i.text.split('\n\n\n')[1].split('\n\n')[:-1]] for i in soup.find_all('div', {'class': 'product-tags__block'})]

In [84]:
tags = dict(zip(tags_names, tags_items))
tags

{'Вкус': ['Минеральное',
  'Долгое послевкусие',
  'Белые фрукты',
  'Долгое',
  'Цитрусовое',
  'Специи'],
 'Гастрономия': ['Закуски, салаты и антипасто', 'Морепродукты', 'Рыба']}

In [81]:
soup.find_all('div', {'class': 'product-tags__block'})[0].text.split('\n\n\n')[1].split('\n\n')[:-1]

['                        Минеральное                    ',
 '                        Долгое послевкусие                    ',
 '                        Белые фрукты                    ',
 '                        Долгое                    ',
 '                        Цитрусовое                    ',
 '                        Специи                    ']

In [19]:
for bottle in tqdm(alco_data_result_copy[1868:]):
    add_info(bottle)
    time.sleep(0.01)

  0%|          | 0/3644 [00:00<?, ?it/s]

In [20]:
df_new = pd.DataFrame(alco_data_result_copy)

Всего каких-то 3,5 часа и датасет собран)

Экспортируем данные в эксель таблицу, чтобы можно было с ними ознакомиться и начать анализ.

In [21]:
df_new.to_csv('/Users/cernenkostanislav/Desktop/Анализ на питон/Алкопарсингновый.csv')